In [1]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!{sys.executable} -m pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!{sys.executable} -m conda install selenium

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::dask-mpi==2022.4.0=pyh458ca06_2
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::dask-mpi==2022.4.0=pyh458ca06_2
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                   /failed

UnsatisfiableError: The following specifications were found to be incompatible with a past
explicit spec that

In [1]:
import os
import sys
os.path.dirname(sys.executable)

'/global/common/software/nersc/pm-2022q2/sw/python/3.9-anaconda-2021.11/bin'

In [2]:
from bs4 import BeautifulSoup
import collections

import sys
if sys.version_info[0] < 3:
    from StringIO import StringIO
    
else:
    from io import StringIO
    
import pandas as pd
from time import sleep

bands = ["g", "r", "i", "z"]

In [3]:
with open("candidate_list.html", "r") as file:
    html = file.read()

soup = BeautifulSoup(html)

In [4]:
def find_candidates(soup):
    rows = soup.body.find_all("tr")
    print(len(rows))
    candidates = []
    # print(rows[2])
    candidates = []
    for i in range(2, len(rows)-2):
        # x1 = float(row[i].attrs["y"])
        s = BeautifulSoup(str(rows[i]))
        a = s.find("a")
        candidates.append(a.text)
    df = pd.DataFrame({"Candidates": candidates})
    df.to_csv("candidates_list.csv")
    
    return df
    
find_candidates(soup)

370


,Candidates
0,DC21badcl
1,DC21bamrg
2,DC21batdg
3,DC21bcmoy
4,DC21bcqwl
...,...
361,DC22fatmk
362,DC22fazip
363,DC22fhhpf
364,DC22fjylx


In [5]:
import psycopg2
import psycopg2.extras
import getpass
from IPython.display import display
import ipywidgets

dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

DB User:  garima
DB Password:  ············


In [8]:
"""
RUN INITIAL CHECK TO SEE WHICH CANDIDATES PASS THE INITIAL CUTS
"""
from astropy.stats import LombScargle
import numpy as np

df = pd.read_csv("candidates_list.csv")
candidates = df["Candidates"]
bands = ["g", "r", "i", "z"]
final_list = []
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )
tolerance = 4

for c in candidates[:]:
    print("ANALYZING CANDIDATE " + c)
    frequency_min = 1/1000
    frequency_max = 1/0.05
    count = 0
    
    for band in bands:
        query = ("SELECT e.mjd, o.mag, o.magerr FROM objects o INNER JOIN candidates c ON " 
                 "o.candidate_id=c.id INNER JOIN subtractions s ON o.subtraction_id=s.id " 
                 "INNER JOIN exposures e ON s.exposure_id=e.id WHERE c.id='" 
                 + c + "' AND e.filter='" + band + "'")

        cursor.execute(query)

        mjd = []
        mag = []
        magerr = []

        # print( f'{"MJD":10s}  {"    MAG":10s}  {"    MAGERR":10s}' )
        # print( '================================================' )
        for row in cursor:
            # print(row["mjd"], row["mag"], row["magerr"])
            mjd.append(row["mjd"])
            mag.append(row["mag"])
            magerr.append(row["magerr"])
        
        mjd = np.array(mjd)
        mag = np.array(mag)
        magerr = np.array(magerr)
        
        try:
            ls = LombScargle(mjd, mag, magerr)
            frequency, power = ls.autopower(method='slow', minimum_frequency = frequency_min, maximum_frequency = frequency_max)
            
            if ls.false_alarm_probability(power[np.argmax(power)]) <= 10**(-10):  # SEE IF ALL BANDS HAVE A FAP < 10^-10 HIGHEST PEAK
                if max(power) > 0.25: # SEE IF ALL BANDS HAVE POWER > 0.25 HIGHEST PEAK
                    print("CANDIDATE " + c + " PASSED THE TESTS FOR BAND " + band)
                    count = count + 1
                
        except ValueError:  #raised if `y` is empty.
            pass
        
    if count >= tolerance:
        final_list.append(c)
        print("PASSED TOLERANCE LEVEL!")
        



In [39]:
print("PERCENTAGE OF CANDIDATES THAT PASSED INITIAL TEST: " + str(len(final_list)*100/len(candidates)) + "%")

PERCENTAGE OF CANDIDATES THAT PASSED INITIAL TEST: 25.40983606557377%


In [42]:
pd.DataFrame({"Candidate": final_list}).to_csv("Updated Candidate List.csv")

93


In [41]:
for c in final_list[15:45]:
    df = pd.DataFrame()
    for band in bands:
        query = ("SELECT e.mjd, o.mag, o.magerr FROM objects o INNER JOIN candidates c ON " 
                 "o.candidate_id=c.id INNER JOIN subtractions s ON o.subtraction_id=s.id " 
                 "INNER JOIN exposures e ON s.exposure_id=e.id WHERE c.id='" 
                 + c + "' AND e.filter='" + band + "'")

        cursor.execute(query)

        mjd = []
        mag = []
        magerr = []

        # print( f'{"MJD":10s}  {"    MAG":10s}  {"    MAGERR":10s}' )
        # print( '================================================' )
        for row in cursor:
            # print(row["mjd"], row["mag"], row["magerr"])
            mjd.append(row["mjd"])
            mag.append(row["mag"])
            magerr.append(row["magerr"])
        
        mjd = np.array(mjd)
        mag = np.array(mag)
        magerr = np.array(magerr)
        
        dfband = pd.DataFrame({band + " MJD":mjd, band + " MAG":mag, band + " MAGERR":magerr})
    
        df = pd.concat([df, dfband], axis=1)
        
    df.to_csv("data/" + c + " 1.csv")
    print("Done!")

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
